# Correlation between flows at Chavuma and Ngonye

## Libraries

In [1]:
import numpy as np
import pandas as pd
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [2]:
ngonye_data='./output_data/2020/'
chavuma_data='./chavuma/output_data/'

## Load the Data

In [3]:
ngo = pd.read_csv(ngonye_data + 'ngonye_daily.csv').set_index('Date')
chav = pd.read_csv(chavuma_data + 'chavuma_daily.csv').set_index('Date')

In [4]:
chav

,Month,Day,Year,WaterYear,Flow,MonthId,WaterMonth,Volume,Exceedance
Date,,,,,,,,,
1959-10-10,10,10,1959,1959.0,49.0,1959.10,1,0.004234,0.957
1959-10-11,10,11,1959,1959.0,49.0,1959.10,1,0.004234,0.957
1959-10-12,10,12,1959,1959.0,49.0,1959.10,1,0.004234,0.957
1959-10-13,10,13,1959,1959.0,49.0,1959.10,1,0.004234,0.957
1959-10-14,10,14,1959,1959.0,47.0,1959.10,1,0.004061,0.963
...,...,...,...,...,...,...,...,...,...
2020-06-16,6,16,2020,2019.0,480.0,2020.06,9,0.041472,0.357
2020-06-17,6,17,2020,2019.0,470.0,2020.06,9,0.040608,0.361
2020-06-18,6,18,2020,2019.0,461.0,2020.06,9,0.039830,0.365


In [5]:
combined=pd.DataFrame({'Chavuma': chav['Flow'],'Chavuma_Exceedance': chav['Exceedance'],'Ngonye': ngo['Flow'],'Ngonye_Exceedance': ngo['Exceedance'],}).dropna(0)
combined

,Chavuma,Chavuma_Exceedance,Ngonye,Ngonye_Exceedance
1959-10-10,49.0,0.957,220.239392,0.924
1959-10-11,49.0,0.957,214.776738,0.936
1959-10-12,49.0,0.957,213.467430,0.939
1959-10-13,49.0,0.957,213.467430,0.939
1959-10-14,47.0,0.963,213.467430,0.939
...,...,...,...,...
2020-05-16,1022.0,0.208,3580.983662,0.056
2020-05-17,991.0,0.213,3559.103900,0.057
2020-05-18,963.0,0.217,3490.510716,0.059
2020-05-19,934.0,0.223,3390.523841,0.063


In [6]:
training, testing  = combined.iloc[:15000, :], combined.iloc[15000:, :]
training, testing

(              Chavuma  Chavuma_Exceedance      Ngonye  Ngonye_Exceedance
 1959-10-10  49.000000               0.957  220.239392              0.924
 1959-10-11  49.000000               0.957  214.776738              0.936
 1959-10-12  49.000000               0.957  213.467430              0.939
 1959-10-13  49.000000               0.957  213.467430              0.939
 1959-10-14  47.000000               0.963  213.467430              0.939
 ...               ...                 ...         ...                ...
 2000-10-29  37.391946               0.990  152.091167              0.994
 2000-10-30  38.418211               0.985  149.404467              0.995
 2000-10-31  38.246318               0.985  148.338383              0.995
 2000-11-01  37.903549               0.989  148.188813              0.995
 2000-11-02  38.074763               0.985  147.656688              0.995
 
 [15000 rows x 4 columns],
                 Chavuma  Chavuma_Exceedance       Ngonye  Ngonye_Exceedance
 2000-

In [7]:
shifts=pd.DataFrame({'Shift':np.arange(1,30,1)})
shifts['All']=shifts.apply(lambda x: training['Chavuma'].shift(x['Shift']).dropna(0).corr(training['Ngonye']),1)
shifts

,Shift,All
0,1,0.798989
1,2,0.810635
2,3,0.821969
3,4,0.832950
4,5,0.843540
5,6,0.853701
6,7,0.863389
7,8,0.872557
8,9,0.881142
9,10,0.889090


In [ ]:
shifts.to_csv(chavuma_data + '')

In [8]:
shifts=np.arange(1,30,1) #
exceeds=[(0,50),(50,100)]#np.arange(20,120,20)

correl=pd.DataFrame({'Shift':shifts})
correl=pd.concat([correl,pd.DataFrame(columns=exceeds)]).set_index('Shift')

In [9]:
for shift in shifts:
    shifted=pd.concat([training[['Chavuma','Chavuma_Exceedance']].shift(shift).dropna(),training[['Ngonye','Ngonye_Exceedance']]],1).dropna()

    for exceed in exceeds:
        days=shifted[(shifted['Chavuma_Exceedance']*100 >= exceed[0]) & (shifted['Chavuma_Exceedance']*100 < exceed[1])]
        correl.loc[shift,exceed]=days['Chavuma'].corr(days['Ngonye'])

correl

,"(0, 50)","(50, 100)"
Shift,,
1.0,0.699475,0.883649
2.0,0.716882,0.889013
3.0,0.733879,0.893352
4.0,0.750398,0.896423
5.0,0.766378,0.898008
6.0,0.781753,0.898319
7.0,0.796448,0.897168
8.0,0.810386,0.89455
9.0,0.823461,0.890438
